<a href="https://colab.research.google.com/github/iamsanthoshg/colabFiles/blob/main/C_R_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install langchain langchain_groq langchain_community chromadb huggingface_hub

In [ ]:
!pip install sentence_transformers

In [19]:
# Import necessary libraries
import os
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryMemory

In [5]:
# 🔹 Step 1: Set Groq API Key
os.environ["GROQ_API_KEY"] = "myGroqApiKey"  # Replace with your actual API key

In [ ]:
# 🔹 Step 2: Load Hugging Face Embeddings
print("Loading Hugging Face embeddings...")
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Embedding model loaded successfully.\n")

In [21]:
# 🔹 Step 3: Load and Process Documents
print("Loading document...")
loader = TextLoader("speech.txt")  # Make sure speech.txt is in the same directory
documents = loader.load()
print(f"Document loaded successfully: {len(documents)} document(s) found.\n")

# 🔹 Step 4: Split Documents into Chunks
print("Splitting document into smaller chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
print(f"Document splitting completed: {len(docs)} chunks created.\n")

# 🔹 Step 5: Create Vector Database (Chroma)
print("Creating vector database with ChromaDB...")
vector_db = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./chroma_db")
retriever = vector_db.as_retriever(search_kwargs={"k": 2})  # Retrieve top 5 matches
print("Vector database created and persisted successfully.\n")

# 🔹 Step 6: Load LLM (Groq API)
print("Loading Mistral model from Groq API...")
llm = ChatGroq(model_name="llama3-8b-8192", groq_api_key=os.environ["GROQ_API_KEY"])
print("Mistral model loaded successfully.\n")

# 🔹 Step 7: Add Conversational Memory
print("Initializing conversational memory...")
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
print("Memory initialized successfully.\n")

# 🔹 Step 8: Create Conversational Retrieval Chain
print("Initializing Conversational Retrieval Chain...")
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    output_key="answer",
    verbose=True
)
print("Conversational Retrieval Chain initialized successfully.\n")

# 🔹 Step 9: Run Interactive Chat Loop
print("\n===== Chatbot Ready! Start asking questions =====")
while True:
    query = input("\nYou: ")

    if query.lower() in ["exit", "quit", "stop"]:
        print("Exiting chatbot. Goodbye!")
        break

    response = conversation_chain.invoke({"question": query})

    # Display Answer
    print("\n🤖 Bot:", response["answer"])

    chat_history = memory.load_memory_variables({})
    print("\n📝 Chat History:", chat_history["chat_history"])


Loading document...
Document loaded successfully: 1 document(s) found.

Splitting document into smaller chunks...
Document splitting completed: 10 chunks created.

Creating vector database with ChromaDB...
Vector database created and persisted successfully.

Loading Mistral model from Groq API...
Mistral model loaded successfully.

Initializing conversational memory...
Memory initialized successfully.

Initializing Conversational Retrieval Chain...
Conversational Retrieval Chain initialized successfully.


===== Chatbot Ready! Start asking questions =====

You: hi


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearan